In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [6]:
shortforms = ['PGP', 'PGPs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [56]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [57]:
list(zip(longforms, counts))

[('p glycoprotein', 81),
 ('proline glycine proline', 57),
 ('plant growth promoting', 48),
 ('pro gly pro', 19),
 ('phosphoglycolate phosphatase', 4),
 ('personal genome project', 3),
 ('pg phosphate', 3),
 ('permeability glycoprotein', 3),
 ('pelvic girdle pain', 2),
 ('panax ginseng polysaccharide', 2),
 ('presynaptic glutamate potential', 2),
 ('pattern generating potential', 2),
 ('peach gum polysaccharide', 2),
 ('phenogenetic project', 2),
 ('protein glycoprotein', 2),
 ('pelgipeptins', 2),
 ('pgp19 phosphoglycoprotein', 1),
 ('porous gelatin particles', 1),
 ('protein gene product', 1),
 ('pulicaria gnaphalodes powder', 1),
 ('prolil glycil proline', 1),
 ('generating phosphatidylglycerol phosphate', 1),
 ('phosphate generating phosphatidylglycerolphosphate', 1),
 ('prevotella glycoprotein', 1),
 ('glycoprotein p', 1),
 ('phophatidylglycerol phosphate', 1),
 ('plant growth promoting abilities', 1),
 ('plant growth promoting traits', 1),
 ('polysaccharide from gynostemma pentaph

In [9]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [10]:
names

{'HGNC:40': 'ABCB1',
 'MESH:C113164': 'prolyl-glycyl-proline',
 'CHEBI:CHEBI:26020': 'phosphate',
 'CHEBI:CHEBI:17089': 'glycoprotein',
 'MESH:D059388': 'Pelvic Girdle Pain',
 'CHEBI:CHEBI:18154': 'polysaccharide',
 'CHEBI:CHEBI:17264': 'phosphatidylglycerol phosphate'}

In [11]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 18:15:11] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [12]:
result = [grounding_map, names, pos_labels]

In [13]:
result

[{'p glycoprotein': 'HGNC:40',
  'panax ginseng polysaccharide': 'ungrounded',
  'pattern generating potential': 'ungrounded',
  'peach gum polysaccharide': 'ungrounded',
  'pelgipeptins': 'ungrounded',
  'pelvic girdle pain': 'MESH:D059388',
  'permeability glycoprotein': 'CHEBI:CHEBI:17089',
  'personal genome project': 'ungrounded',
  'pg phosphate': 'ungrounded',
  'phenogenetic project': 'ungrounded',
  'phosphoglycolate phosphatase': 'HGNC:8909',
  'plant growth promoting': 'ungrounded',
  'presynaptic glutamate potential': 'ungrounded',
  'pro gly pro': 'MESH:C428643',
  'proline glycine proline': 'MESH:C428643',
  'protein glycoprotein': 'ungrounded'},
 {'HGNC:40': 'ABCB1',
  'MESH:D059388': 'Pelvic Girdle Pain',
  'CHEBI:CHEBI:17089': 'glycoprotein',
  'HGNC:8909': 'PGP',
  'MESH:C428643': 'N-acetyl-proline-glycine-proline'},
 ['HGNC:40', 'HGNC:8909']]

In [61]:
grounding_map, names, pos_labels = [{'p glycoprotein': 'HGNC:40',
  'panax ginseng polysaccharide': 'ungrounded',
  'pattern generating potential': 'ungrounded',
  'peach gum polysaccharide': 'ungrounded',
  'pelgipeptins': 'ungrounded',
  'pelvic girdle pain': 'MESH:D059388',
  'permeability glycoprotein': 'CHEBI:CHEBI:17089',
  'personal genome project': 'ungrounded',
  'pg phosphate': 'HGNC:8909',
  'phenogenetic project': 'ungrounded',
  'phosphoglycolate phosphatase': 'HGNC:8909',
  'plant growth promoting': 'ungrounded',
  'presynaptic glutamate potential': 'ungrounded',
  'pro gly pro': 'MESH:C428643',
  'proline glycine proline': 'MESH:C428643',
  'protein glycoprotein': 'ungrounded',
  'phosphatidylglycerolphosphate': 'HGNC:8909',
  'phophatidylglycerol phosphate': 'HGNC:8909'},
 {'HGNC:40': 'ABCB1',
  'MESH:D059388': 'Pelvic Girdle Pain',
  'CHEBI:CHEBI:17089': 'glycoprotein',
  'HGNC:8909': 'PGP',
  'MESH:C428643': 'N-acetyl-proline-glycine-proline'},
 ['HGNC:40', 'HGNC:8909']]

In [62]:
excluded_longforms = []

In [63]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [64]:
additional_entities = {'HGNC:8909': ['PGP', ['PGP', 'phosphoglycolate']]}

In [65]:
unambiguous_agent_texts = {}

In [66]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [67]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [68]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [69]:
intersection1

[('HGNC:8909', 'HGNC:8909', 25)]

In [70]:
intersection2

[('HGNC:40', 'HGNC:8909', 0),
 ('ungrounded', 'HGNC:8909', 0),
 ('HGNC:8909', 'HGNC:8909', 0),
 ('MESH:C428643', 'HGNC:8909', 0),
 ('MESH:D059388', 'HGNC:8909', 0),
 ('CHEBI:CHEBI:17089', 'HGNC:8909', 0)]

In [71]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [72]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [73]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 18:35:33] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 18:35:36] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8821520829739521 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [74]:
classifier.stats

{'label_distribution': {'HGNC:40': 73,
  'ungrounded': 42,
  'HGNC:8909': 12,
  'MESH:C428643': 50,
  'MESH:D059388': 2,
  'CHEBI:CHEBI:17089': 1},
 'f1': {'mean': 0.882152, 'std': 0.074321},
 'precision': {'mean': 0.848039, 'std': 0.101335},
 'recall': {'mean': 0.939624, 'std': 0.039627},
 'ungrounded': {'f1': {'mean': 0.938137, 'std': 0.039635},
  'pr': {'mean': 0.905556, 'std': 0.047629},
  'rc': {'mean': 0.975, 'std': 0.05}},
 'CHEBI:CHEBI:17089': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:8909': {'f1': {'mean': 0.626667, 'std': 0.336254},
  'pr': {'mean': 0.533333, 'std': 0.323179},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'HGNC:40': {'f1': {'mean': 0.919282, 'std': 0.042289},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.853595, 'std': 0.076036}},
 'MESH:D059388': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:C428643': {'f1': {'mean': 0.968421

In [75]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [76]:
disamb.dump(model_name, results_path)

In [77]:
print(disamb.info())

Disambiguation model for PGP, and PGPs

Produces the disambiguations:
	ABCB1*	HGNC:40
	N-acetyl-proline-glycine-proline	MESH:C428643
	PGP*	HGNC:8909
	Pelvic Girdle Pain	MESH:D059388
	glycoprotein	CHEBI:CHEBI:17089

Class level metrics:
--------------------
Grounding                       	Count	F1     
                           ABCB1*	73	0.91928
N-acetyl-proline-glycine-proline	50	0.96842
                      Ungrounded	42	0.93814
                             PGP*	12	0.62667
              Pelvic Girdle Pain	 2	    0.2
                    glycoprotein	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.88215
	Precision:	0.84804
	Recall:		0.93962

* Positive labels
See Docstring for explanation



In [78]:
model_to_s3(disamb)

In [46]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [47]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:8909']

In [55]:
texts[5]

IndexError: list index out of range